Read in and check null values.

In [125]:
import pandas as pd

In [126]:
historical_weather_df = pd.read_csv("historical_weather_data.csv")
null_counts = historical_weather_df.isnull().sum()
print(null_counts)

timestamp             0
temperature_f         0
dew_point_f           0
humidity_%            0
wind                 10
wind_speed_mph        0
wind_gust_mph         0
pressure_in           0
precip_in             0
weather_condition     0
datetime              0
dtype: int64


Remove all the null values records.

In [127]:
historical_weather_df['wind'] = historical_weather_df['wind'].fillna('MISSING')
historical_weather_df.head(5)

,timestamp,temperature_f,dew_point_f,humidity_%,wind,wind_speed_mph,wind_gust_mph,pressure_in,precip_in,weather_condition,datetime
0,1676508660,56,50,80,S,3,0,29.79,0.0,Cloudy,2023-02-16 00:51:00
1,1676512260,61,39,44,SW,16,24,29.91,0.0,Partly Cloudy,2023-02-16 01:51:00
2,1676515860,61,39,44,WSW,16,23,29.93,0.0,Mostly Cloudy,2023-02-16 02:51:00
3,1676519460,60,38,44,WSW,15,0,29.93,0.0,Mostly Cloudy,2023-02-16 03:51:00
4,1676523060,60,38,44,SW,13,0,29.94,0.0,Mostly Cloudy,2023-02-16 04:51:00


Remove columns that won't be used.

In [128]:
historical_weather_df = historical_weather_df.drop(columns=['dew_point_f', 'precip_in'])

Change units.

In [129]:
historical_weather_df['temp'] = (historical_weather_df['temperature_f'] - 32) * 5 / 9 + 273.15
historical_weather_df = historical_weather_df.drop(columns=['temperature_f'])

historical_weather_df['wind_speed'] = historical_weather_df['wind_speed_mph'] * 0.44704
historical_weather_df = historical_weather_df.drop(columns=['wind_speed_mph'])

historical_weather_df['wind_gust'] = historical_weather_df['wind_gust_mph'] * 0.44704
historical_weather_df = historical_weather_df.drop(columns=['wind_gust_mph'])

historical_weather_df['pressure'] = historical_weather_df['pressure_in'] * 33.8639
historical_weather_df = historical_weather_df.drop(columns=['pressure_in'])

historical_weather_df = historical_weather_df.rename(columns={'wind': 'wind_direction', 'humidity_%': 'humidity'})

historical_weather_df.head(5)

,timestamp,humidity,wind_direction,weather_condition,datetime,temp,wind_speed,wind_gust,pressure
0,1676508660,80,S,Cloudy,2023-02-16 00:51:00,286.483333,1.34112,0.00000,1008.805581
1,1676512260,44,SW,Partly Cloudy,2023-02-16 01:51:00,289.261111,7.15264,10.72896,1012.869249
2,1676515860,44,WSW,Mostly Cloudy,2023-02-16 02:51:00,289.261111,7.15264,10.28192,1013.546527
3,1676519460,44,WSW,Mostly Cloudy,2023-02-16 03:51:00,288.705556,6.70560,0.00000,1013.546527
4,1676523060,44,SW,Mostly Cloudy,2023-02-16 04:51:00,288.705556,5.81152,0.00000,1013.885166


Weather condition coding to OpenWeatherAPI.

In [130]:
weather_conditions = historical_weather_df['weather_condition'].unique()
print(weather_conditions)

['Cloudy' 'Partly Cloudy' 'Mostly Cloudy' 'Fair' 'Light Rain'
 'Mostly Cloudy / Windy' 'Rain / Windy' 'Heavy Rain / Windy'
 'Light Rain / Windy' 'Cloudy / Windy' 'Wintry Mix / Windy'
 'Partly Cloudy / Windy' 'Fair / Windy' 'Rain' 'Light Rain with Thunder'
 'T-Storm' 'Wintry Mix' 'Light Drizzle' 'Fog' 'Haze' 'Light Snow'
 'Light Snow and Sleet' 'Snow and Sleet' 'Snow' 'Light Snow / Windy'
 'Light Drizzle / Windy' 'T-Storm / Windy' 'Heavy Rain' 'Drizzle and Fog'
 'Thunder' 'Thunder in the Vicinity' 'Haze / Windy' 'Smoke / Windy'
 'Smoke' 'Heavy T-Storm' 'Mist' 'Heavy T-Storm / Windy' 'Heavy Rain / Fog'
 'Mist / Windy' 'Light Freezing Rain' 'Snow / Fog' 'Heavy Snow / Fog'
 'Light Rain / Fog' 'Thunder and Hail / Fog' 'Fog / Windy']


We have read and connected the weather condition expressions to those of OpenWeatherAPI.

In [131]:
mapping_df = pd.read_csv("weather_mapping.csv")
weather_id_dict = mapping_df.set_index('Description')['Weather ID'].to_dict()
historical_weather_df['weather_id'] = historical_weather_df['weather_condition'].map(weather_id_dict)
historical_weather_df.head(5)

,timestamp,humidity,wind_direction,weather_condition,datetime,temp,wind_speed,wind_gust,pressure,weather_id
0,1676508660,80,S,Cloudy,2023-02-16 00:51:00,286.483333,1.34112,0.00000,1008.805581,804
1,1676512260,44,SW,Partly Cloudy,2023-02-16 01:51:00,289.261111,7.15264,10.72896,1012.869249,801
2,1676515860,44,WSW,Mostly Cloudy,2023-02-16 02:51:00,289.261111,7.15264,10.28192,1013.546527,803
3,1676519460,44,WSW,Mostly Cloudy,2023-02-16 03:51:00,288.705556,6.70560,0.00000,1013.546527,803
4,1676523060,44,SW,Mostly Cloudy,2023-02-16 04:51:00,288.705556,5.81152,0.00000,1013.885166,803


Process wind_direction

In [132]:
print(historical_weather_df['wind_direction'].unique())

['S' 'SW' 'WSW' 'SSW' 'CALM' 'NW' 'VAR' 'SSE' 'NNW' 'WNW' 'W' 'ENE' 'E'
 'N' 'NE' 'NNE' 'SE' 'ESE' 'MISSING']


Save.

In [133]:
historical_weather_df.to_csv("weather_processed.csv", index=False)